<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/PGM_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
!pip install openpyxl
!pip install logging
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.3 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import logging
import argparse
from collections import defaultdict
from datetime import datetime, timedelta
from openpyxl import load_workbook
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Constants
URL = "https://matthey.com/products-and-markets/pgms-and-circularity/pgm-management?p_p_id=jm_metal_price_table_portlet_MetalPriceTablePortlet&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage"
TROY_TO_GRAM = 31.1035
# SAVE_FOLDER = r"C:\Users\User\Documents\PGM-Monthly-Report"
SAVE_FOLDER = "test/"

METALS = {"Platinum": "Pt", "Rhodium": "Rh"}
MONTH_DICT = {
    "Jan": "1", "Feb": "2", "Mar": "3", "Apr": "4", "May": "5", "Jun": "6",
    "Jul": "7", "Aug": "8", "Sep": "9", "Oct": "10", "Nov": "11", "Dec": "12"
}

def parse_start_date():
    parser = argparse.ArgumentParser(description="Fetch PGM Prices from a specified month and year until the current month.")
    parser.add_argument("start_date", nargs="?", default=None, help="Start date in MM-DD-YYYY format (e.g., '05-01-2024')")
    args = parser.parse_args()

    if args.start_date:
        try:
            input_date = datetime.strptime(args.start_date, "%m-%d-%Y")
            return input_date.replace(day=1)
        except ValueError:
            logging.error("Invalid date format. Please use 'MM-DD-YYYY'. Falling back to default.")

    return datetime(datetime.today().year, 1, 1)

def fetch_pgm_prices(date, metal):
    date_str = date.strftime("%d-%m-%Y")
    payload = {
        "_jm_metal_price_table_portlet_MetalPriceTablePortlet_priceRadio": metal,
        "_jm_metal_price_table_portlet_MetalPriceTablePortlet_pricetableDate": date_str
    }
    try:
        logging.info(f"Fetching {metal} data for {date.strftime('%B %Y')}...")
        response = requests.post(URL, data=payload)
        response.raise_for_status()
        logging.info("Data successfully fetched from API.")
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching {metal} data: {e}")
        return None

def process_data(data, date, metal):
    if not data:
        logging.error(f"No data to process for {metal}.")
        return None

    logging.info(f"Processing {metal} data for {date.strftime('%B %Y')}...")
    time_zone_market_avg_list = data.get("timeZoneMarketAvglist", "")

    return {
        f"{metal} (Troy)": time_zone_market_avg_list,
        f"{metal} (gr)": float(time_zone_market_avg_list) / TROY_TO_GRAM if time_zone_market_avg_list else None
    }

def transform_Last5DaysPrices(data):
    # Use defaultdict to collect prices by date
    result_dict = defaultdict(dict)

    for entry in data:
        date = entry['Date']
        metal = entry['Metal'].lower()
        unit = entry['Unit'].lower()
        price = entry['Price ($)']

        key = f"{metal}_price_per_{unit}"
        result_dict[date][key] = price

    # Convert to the desired list of dictionaries
    final_result = []
    for date, prices in result_dict.items():
        combined_entry = {'date': date}
        combined_entry.update(prices)
        final_result.append(combined_entry)

    return final_result

def save_to_excel(filepath, monthly_df, last5_df):
    with pd.ExcelWriter(filepath, engine="xlsxwriter") as writer:
        monthly_df.to_excel(writer, sheet_name="All-Monthly-Averages", index=False)
        last5_df.to_excel(writer, sheet_name="Last-5-Days", index=False)

def main():
    logging.info("Starting PGM price fetching script...")

    # Ensure save folder exists
    os.makedirs(SAVE_FOLDER, exist_ok=True)

    # start_date = parse_start_date()
    start_date = datetime.strptime("01-01-2022", "%m-%d-%Y")
    current_date = datetime.today().replace(day=1) - timedelta(days=1)
    last_month_str = current_date.strftime('%b-%y')
    last_month_label = current_date.strftime('%b%Y')  # ← updated to use LAST processed month

    data_list = []
    last_five_days_prices = []

    while start_date <= current_date:
        row_data = {"Date": start_date.strftime('%m-%d-%Y')}

        for metal_name, metal_code in METALS.items():
            data = fetch_pgm_prices(start_date, metal_code)
            processed_data = process_data(data, start_date, metal_name)
            # print(processed_data)
            if processed_data:
                row_data.update(processed_data)
                print(row_data)

            if start_date.strftime('%b-%y') == last_month_str and data:
                table_list = data.get("tableList", {}).get("columns", [])
                last_5 = {entry["IntervalDate"]: entry["Price3"] for entry in table_list[-5:]}

                for date, price in last_5.items():
                    try:
                        formatted_date = date.split(", ")[1]
                        day, month, year = formatted_date.split(" ")
                        formatted_date_str = f"{day}-{MONTH_DICT.get(month, month)}-{year}"

                        last_five_days_prices.append({
                            "Date": formatted_date_str,
                            "Metal": metal_name,
                            "Unit": "Troy",
                            "Price ($)": float(price) if price else None
                        })
                        last_five_days_prices.append({
                            "Date": formatted_date_str,
                            "Metal": metal_name,
                            "Unit": "Gram",
                            "Price ($)": float(price) / TROY_TO_GRAM if price else None
                        })
                    except Exception as e:
                        logging.warning(f"Error formatting date {date}: {e}")
        # print(row_data)
        data_list.append(row_data)
        start_date = (start_date.replace(day=1) + timedelta(days=32)).replace(day=1)

    MonthlyAverages_df = pd.DataFrame(data_list)
    # MonthlyAverages_df = MonthlyAverages_df[["Date", "Platinum (Troy)", "Platinum (gr)", "Date", "Rhodium (Troy)", "Rhodium (gr)"]]
    Last5Days = transform_Last5DaysPrices(last_five_days_prices)
    Last5Days_df = pd.DataFrame(Last5Days)

    # Build file paths
    master_file_path = os.path.join(SAVE_FOLDER, "master.xlsx")
    month_file_path = os.path.join(SAVE_FOLDER, f"PGM-{last_month_label}.xlsx")

    save_to_excel(master_file_path, MonthlyAverages_df, Last5Days_df)
    save_to_excel(month_file_path, MonthlyAverages_df, Last5Days_df)

    logging.info(f"Files saved to folder: {SAVE_FOLDER}")
    logging.info("Script execution completed.")
    print(MonthlyAverages_df)

In [ ]:
if __name__ == "__main__":
    main()

{'Date': '01-01-2022', 'Platinum (Troy)': '998.4878', 'Platinum (gr)': 32.102104264793354}
{'Date': '01-01-2022', 'Platinum (Troy)': '998.4878', 'Platinum (gr)': 32.102104264793354, 'Rhodium (Troy)': '16477.4390', 'Rhodium (gr)': 529.7615702412911}
{'Date': '02-01-2022', 'Platinum (Troy)': '1056.4459', 'Platinum (gr)': 33.96549905959136}
{'Date': '02-01-2022', 'Platinum (Troy)': '1056.4459', 'Platinum (gr)': 33.96549905959136, 'Rhodium (Troy)': '18183.4459', 'Rhodium (gr)': 584.6109248155352}
{'Date': '03-01-2022', 'Platinum (Troy)': '1053.7174', 'Platinum (gr)': 33.8777758130114}
{'Date': '03-01-2022', 'Platinum (Troy)': '1053.7174', 'Platinum (gr)': 33.8777758130114, 'Rhodium (Troy)': '19402.1739', 'Rhodium (gr)': 623.793910653142}
{'Date': '04-01-2022', 'Platinum (Troy)': '973.1200', 'Platinum (gr)': 31.286511164338418}
{'Date': '04-01-2022', 'Platinum (Troy)': '973.1200', 'Platinum (gr)': 31.286511164338418, 'Rhodium (Troy)': '18857.3333', 'Rhodium (gr)': 606.2768916681401}
{'Date'

In [ ]:
from collections import defaultdict

def transform_Last5DaysPrices(data):
    # Use defaultdict to collect prices by date
    result_dict = defaultdict(dict)

    for entry in data:
        date = entry['Date']
        metal = entry['Metal'].lower()
        unit = entry['Unit'].lower()
        price = entry['Price ($)']

        key = f"{metal}_price_per_{unit}"
        result_dict[date][key] = price

    # Convert to the desired list of dictionaries
    final_result = []
    for date, prices in result_dict.items():
        combined_entry = {'date': date}
        combined_entry.update(prices)
        final_result.append(combined_entry)

    return final_result

In [ ]:
test = [{'Date': '25-3-2025', 'Metal': 'Platinum', 'Unit': 'Troy', 'Price ($)': 992.0}, {'Date': '25-3-2025', 'Metal': 'Platinum', 'Unit': 'Gram', 'Price ($)': 31.89351680679023}, {'Date': '26-3-2025', 'Metal': 'Platinum', 'Unit': 'Troy', 'Price ($)': 986.0}, {'Date': '26-3-2025', 'Metal': 'Platinum', 'Unit': 'Gram', 'Price ($)': 31.70061247126529}, {'Date': '27-3-2025', 'Metal': 'Platinum', 'Unit': 'Troy', 'Price ($)': 979.0}, {'Date': '27-3-2025', 'Metal': 'Platinum', 'Unit': 'Gram', 'Price ($)': 31.47555741315286}, {'Date': '28-3-2025', 'Metal': 'Platinum', 'Unit': 'Troy', 'Price ($)': 995.0}, {'Date': '28-3-2025', 'Metal': 'Platinum', 'Unit': 'Gram', 'Price ($)': 31.989968974552703}, {'Date': '31-3-2025', 'Metal': 'Platinum', 'Unit': 'Troy', 'Price ($)': 993.0}, {'Date': '31-3-2025', 'Metal': 'Platinum', 'Unit': 'Gram', 'Price ($)': 31.925667529377723}, {'Date': '25-3-2025', 'Metal': 'Rhodium', 'Unit': 'Troy', 'Price ($)': 5575.0}, {'Date': '25-3-2025', 'Metal': 'Rhodium', 'Unit': 'Gram', 'Price ($)': 179.2402784252576}, {'Date': '26-3-2025', 'Metal': 'Rhodium', 'Unit': 'Troy', 'Price ($)': 5775.0}, {'Date': '26-3-2025', 'Metal': 'Rhodium', 'Unit': 'Gram', 'Price ($)': 185.67042294275564}, {'Date': '27-3-2025', 'Metal': 'Rhodium', 'Unit': 'Troy', 'Price ($)': 5775.0}, {'Date': '27-3-2025', 'Metal': 'Rhodium', 'Unit': 'Gram', 'Price ($)': 185.67042294275564}, {'Date': '28-3-2025', 'Metal': 'Rhodium', 'Unit': 'Troy', 'Price ($)': 5725.0}, {'Date': '28-3-2025', 'Metal': 'Rhodium', 'Unit': 'Gram', 'Price ($)': 184.06288681338114}, {'Date': '31-3-2025', 'Metal': 'Rhodium', 'Unit': 'Troy', 'Price ($)': 5700.0}, {'Date': '31-3-2025', 'Metal': 'Rhodium', 'Unit': 'Gram', 'Price ($)': 183.25911874869388}]
output = transform_metal_prices(test)
output

[{'date': '25-3-2025',
  'platinum_price_per_troy': 992.0,
  'platinum_price_per_gram': 31.89351680679023,
  'rhodium_price_per_troy': 5575.0,
  'rhodium_price_per_gram': 179.2402784252576},
 {'date': '26-3-2025',
  'platinum_price_per_troy': 986.0,
  'platinum_price_per_gram': 31.70061247126529,
  'rhodium_price_per_troy': 5775.0,
  'rhodium_price_per_gram': 185.67042294275564},
 {'date': '27-3-2025',
  'platinum_price_per_troy': 979.0,
  'platinum_price_per_gram': 31.47555741315286,
  'rhodium_price_per_troy': 5775.0,
  'rhodium_price_per_gram': 185.67042294275564},
 {'date': '28-3-2025',
  'platinum_price_per_troy': 995.0,
  'platinum_price_per_gram': 31.989968974552703,
  'rhodium_price_per_troy': 5725.0,
  'rhodium_price_per_gram': 184.06288681338114},
 {'date': '31-3-2025',
  'platinum_price_per_troy': 993.0,
  'platinum_price_per_gram': 31.925667529377723,
  'rhodium_price_per_troy': 5700.0,
  'rhodium_price_per_gram': 183.25911874869388}]